# Workflow: 

- Step 1.  For 'Position_clean' column, find all rows that contain substring combo ('intern' OR 'co-op) AND ('data science' OR 'data scientist' OR 'ML' or 'machine learning') *case insensitive*
    - replace all with 'Data Scientist Intern' 
- Step 2.  find all rows that contain substring 'data science'*case insensitive*
    - replace case by case with 'xx Data Scientist' 
- Step 3. find all rows that contain substring 'analytics' *case insensitive*
    - replace case by case with 'xx Data Scientist' 
- Step 4. find all rows that contain substring 'ML' or 'machine learning' (assume: exclude 'AI' roles) *case-insensitive*
    - replace case by case with 'xx Data Scientist' 
 
- Step 5. Other special cases (from examining df.Position. unique()): replace case by case with 'xx Data Scientist' 
  
- Step 6: find all rows that do NOT contain substring *case-insensitive* 'Data Scientist'(e.g. 'software engineer', 'AI research scientist', 'data engineer', 'software developer', 'database developer', 'data analyst') -> replace with np.nan (for easier removal of these rows later) 


[see here for str.replace](https://stackoverflow.com/questions/39768547/replace-whole-string-if-it-contains-substring-in-pandas)

**note: order matters! e.g. '*Senior* *director*, data scientist' - need to change to LDS, before SR**

- Step 7A. replace any substrings 'lead' or 'director' or 'manager' or 'principal' -> 'Lead Data Scientist'
- Step 7B. replace any substrings 'sr' or 'senior' -> 'Senior Data Scientist'
- Step 7C. replace any substrings jr' or 'junior' -> 'Junior Data Scientist'
- Step 7D. replace substrings NOT 'intern' NOT 'junior' NOT 'senior' NOT 'lead' NOT NaN  -> 'Data Scientist' 


AFTER merging with Sasha's cleaned columns, do following: 
- drop rows that have NaN in the 'Position_clean' column [see how here](https://stackoverflow.com/questions/13413590/how-to-drop-rows-of-pandas-dataframe-whose-value-in-a-certain-column-is-nan)
- check and drop duplicates
- drop other rows with certain NaN fields

In [34]:
import pandas as pd
import numpy as np

In [2]:
INTERN = 'Data Scientist Intern'
JR = 'Junior Data Scientist'
SR = 'Senior Data Scientist'
DS = 'Data Scientist'
LDS = 'Lead Data Scientist'
rm = np.nan

In [3]:
df_ca = pd.read_csv('jack_canada.csv', index_col = 0)

In [4]:
df_ca.shape

(1258, 12)

In [5]:
df_ca.head()

,Company,Description,Glassdoor Salary Estimate,Industry,Location,Position,Rating,Revenue,Sector,Size,Type,Year Founded
0,AlayaCare,Do you believe that recent advancements in AI ...,NaN,Enterprise Software & Network Solutions,Montreal,Data Scientist,4.4,Unknown / Non-Applicable,Information Technology,201 to 500 Employees,Company - Private,2014.0
1,Special Risk Insurance Managers Ltd.,Be a part of a dynamic and growing team at Spe...,NaN,NaN,Langley,Data Analyst,NaN,NaN,NaN,NaN,NaN,NaN
2,Syniti,Syniti was founded to solve business’ complex ...,NaN,IT Services,Moncton,Data Analyst,3.7,$100 to $500 million (USD),Information Technology,501 to 1000 Employees,Company - Private,1996.0
3,Points International,Company Description\r\r\r\r\r\r\r\r\r\r\r\r\r\...,NaN,Internet,Toronto,Data Scientist,3.6,$100 to $500 million (USD),Information Technology,201 to 500 Employees,Company - Public,2001.0
4,Spin Master Ltd,Are you looking to further your career with an...,NaN,Consumer Products Manufacturing,Toronto,"Senior Manager, Data Science",3.0,$2 to $5 billion (USD),Manufacturing,1001 to 5000 Employees,Company - Public,1994.0


In [6]:
# duplicate
df_ca['Position_clean'] = df_ca.loc[:, 'Position']

In [7]:
df_ca['Position'].unique()

array(['Data Scientist', 'Data Analyst', 'Senior Manager, Data Science',
       'Automotive Software Data Analyst / Tester - Remote',
       'Junior Data Analyst',
       'Optimization Operational Research Scientist',
       'Data Scientist Intern', 'Junior Scientist',
       'Senior Database Developer', 'Master Data Analyst / SAP (Junior)',
       'Data Analyst - Junior', 'Data Analyst (Remote)',
       'Remote PT Data Science Mentor',
       'Remote Cloud Database developer (Python)',
       'Reporting Consultant & Data Analyst / Cognos Expert',
       'Flight Data Analyst', 'Junior Software Developer',
       'Product Data Analyst', 'Data Analyst II',
       'Entry Level Data Analyst (Bilingual French & English)',
       'Jr. Data Analyst', 'Digital Data Analyst',
       'Principal Data Scientist', 'Data Analyst, Security - 297742',
       'Tech - Data Analyst', 'Data Analyst, Product',
       'Data Analyst Intern, Global Investments IT',
       'Data Analyst - Level 1 x2 (Contract)

In [8]:
df_ca[df_ca['Position'] == 'Senior Statistical Programmer/Statistical Scientist']

,Company,Description,Glassdoor Salary Estimate,Industry,Location,Position,Rating,Revenue,Sector,Size,Type,Year Founded,Position_clean
692,IQVIA,IQVIA™ is the leading human data science compa...,NaN,Biotech & Pharmaceuticals,Kirkland,Senior Statistical Programmer/Statistical Scie...,3.7,$2 to $5 billion (USD),Biotech & Pharmaceuticals,10000+ Employees,Company - Public,2017.0,Senior Statistical Programmer/Statistical Scie...


In [9]:
## step 1
s = ['intern', 'co-op', 'co op', 'placem']
cond_or1 = '|'.join(s)
t = ["data scien", "machine learn"]
cond_or2 = '|'.join(t)
bool_intern = (df_ca['Position'].str.contains(cond_or1, case = False))& \
               ( (df_ca['Position'].str.contains(cond_or2, case = False)) |\
                (df_ca['Position'].str.contains('ML', case = True))
               ) 
df_ca[bool_intern] # display rows

,Company,Description,Glassdoor Salary Estimate,Industry,Location,Position,Rating,Revenue,Sector,Size,Type,Year Founded,Position_clean
10,Guavus,"Guavus, a Thales company, is seeking a Data Sc...",NaN,Enterprise Software & Network Solutions,Montreal,Data Scientist Intern,2.8,$25 to $50 million (USD),Information Technology,201 to 500 Employees,Subsidiary or Business Segment,2006.0,Data Scientist Intern
286,Reesby IT,About the Data Scientist Internship\r\r\r\r\r\...,NaN,NaN,British Columbia,Machine Learning Artificial Intelligence Inter...,NaN,NaN,NaN,NaN,NaN,NaN,Machine Learning Artificial Intelligence Inter...
370,Guavus,"Guavus, a Thales company, is seeking a Data Sc...",NaN,Enterprise Software & Network Solutions,Montreal,Data Scientist Intern,2.8,$25 to $50 million (USD),Information Technology,201 to 500 Employees,Subsidiary or Business Segment,2006.0,Data Scientist Intern
1056,RBC,What is the opportunity?\r\r\r\r\r\r\r\r\n\r\r...,NaN,Banks & Credit Unions,Toronto,Data Scientist - Winter 2021 Internship (8 Mon...,4.0,$10+ billion (USD),Finance,10000+ Employees,Company - Public,1869.0,Data Scientist - Winter 2021 Internship (8 Mon...
1159,QuadReal,About QuadReal\r\r\r\r\r\nHeadquartered in Van...,NaN,Real Estate,Toronto,"Intern, Data Scientist",4.4,Unknown / Non-Applicable,Real Estate,1001 to 5000 Employees,Company - Private,2016.0,"Intern, Data Scientist"
1174,Guavus,"Guavus, a Thales company, is seeking a Data Sc...",NaN,Enterprise Software & Network Solutions,Montreal,Data Scientist Intern,2.8,$25 to $50 million (USD),Information Technology,201 to 500 Employees,Subsidiary or Business Segment,2006.0,Data Scientist Intern
1181,"Southern Graphics Systems, Canada Co.",Data Scientist Intern– sgsco labs\r\r\r\r\n\r\...,NaN,Advertising & Marketing,Toronto,Intern - Data Science,3.0,Unknown / Non-Applicable,Business Services,1001 to 5000 Employees,Subsidiary or Business Segment,1946.0,Intern - Data Science
1208,sgsco,Data Scientist Intern– sgsco labs\r\r\r\n\r\r\...,NaN,Advertising & Marketing,Toronto,Intern - Data Science,2.5,Unknown / Non-Applicable,Business Services,1001 to 5000 Employees,Company - Private,1946.0,Intern - Data Science
1210,American Iron and Metal,Company Description\r\r\r\n\r\r\r\nES-TU FORGÉ...,NaN,Metal & Mineral Manufacturing,Montréal-Est,Internship Data Scientist,2.6,$1 to $2 billion (USD),Manufacturing,1001 to 5000 Employees,Company - Private,1936.0,Internship Data Scientist


In [10]:
df_ca.loc[bool_intern, 'Position_clean'] = INTERN
# 1c. check by comparing 2 columns
df_ca[bool_intern][['Position', 'Position_clean']]

,Position,Position_clean
10,Data Scientist Intern,Data Scientist Intern
286,Machine Learning Artificial Intelligence Inter...,Data Scientist Intern
370,Data Scientist Intern,Data Scientist Intern
1056,Data Scientist - Winter 2021 Internship (8 Mon...,Data Scientist Intern
1159,"Intern, Data Scientist",Data Scientist Intern
1174,Data Scientist Intern,Data Scientist Intern
1181,Intern - Data Science,Data Scientist Intern
1208,Intern - Data Science,Data Scientist Intern
1210,Internship Data Scientist,Data Scientist Intern


In [11]:
## Step 2
bool_2 = df_ca['Position'].str.contains('data science', case = False)
df_ca[bool_2] # display

,Company,Description,Glassdoor Salary Estimate,Industry,Location,Position,Rating,Revenue,Sector,Size,Type,Year Founded,Position_clean
4,Spin Master Ltd,Are you looking to further your career with an...,NaN,Consumer Products Manufacturing,Toronto,"Senior Manager, Data Science",3.0,$2 to $5 billion (USD),Manufacturing,1001 to 5000 Employees,Company - Public,1994.0,"Senior Manager, Data Science"
75,Lighthouse Labs,*DUE TO COVID WE ARE CURRENTLY FULLY REMOTE AN...,NaN,Education Training Services,Vancouver,Remote PT Data Science Mentor,4.1,Unknown / Non-Applicable,Education,1 to 50 Employees,Company - Private,2013.0,Remote PT Data Science Mentor
204,Loblaw Companies Limited,Lieu:\r\r\r\r\r\r\r\r\n1 Presidents Choice Cir...,NaN,Vehicle Dealers,Brampton,"Director, Data Science",3.6,$10+ billion (USD),Retail,10000+ Employees,Company - Public,NaN,"Director, Data Science"
210,Loblaw,Location:\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\n1 Pre...,NaN,Vehicle Dealers,Brampton,"Director, Data Science - AIML Coe",3.6,$10+ billion (USD),Retail,10000+ Employees,Company - Public,NaN,"Director, Data Science - AIML Coe"
286,Reesby IT,About the Data Scientist Internship\r\r\r\r\r\...,NaN,NaN,British Columbia,Machine Learning Artificial Intelligence Inter...,NaN,NaN,NaN,NaN,NaN,NaN,Data Scientist Intern
363,Spin Master Ltd,Are you looking to further your career with an...,NaN,Consumer Products Manufacturing,Toronto,"Senior Manager, Data Science",3.0,$2 to $5 billion (USD),Manufacturing,1001 to 5000 Employees,Company - Public,1994.0,"Senior Manager, Data Science"
393,Spin Master Ltd,Are you looking to further your career with an...,NaN,Consumer Products Manufacturing,Toronto,"Senior Manager, Data Science",3.0,$2 to $5 billion (USD),Manufacturing,1001 to 5000 Employees,Company - Public,1994.0,"Senior Manager, Data Science"
395,Bell Canada,Req Id: 272559\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r\r...,NaN,"Cable, Internet & Telephone Providers",Toronto,"Senior Manager, Data Science",3.7,$10+ billion (USD),Telecommunications,10000+ Employees,Company - Public,1880.0,"Senior Manager, Data Science"
459,Citi,The Applications Development Senior Manager ac...,NaN,Investment Banking & Asset Management,Mississauga,Data Science Senior Manager,3.8,$10+ billion (USD),Finance,10000+ Employees,Company - Public,1812.0,Data Science Senior Manager
474,Citibank,The Applications Development Senior Manager ac...,NaN,Lending,Mississauga,Data Science Senior Manager,3.5,$50 to $100 million (USD),Finance,5001 to 10000 Employees,Subsidiary or Business Segment,NaN,Data Science Senior Manager


In [12]:
# special
df_ca.loc[df_ca['Position'] == 'Remote PT Data Science Mentor', 'Position_clean' ] = rm

In [13]:
# lead ds
bool_lds = (df_ca['Position_clean'].str.contains('Manage|Direct|Lead', case = False)) & \
           (df_ca['Position_clean'].str.contains('data science', case = False))
df_ca.loc[bool_lds, 'Position_clean'] = LDS
# sr ds
bool_sr = (df_ca['Position_clean'].str.contains('Senior|Sr', case = False)) &\
            (df_ca['Position_clean'].str.contains('data science', case = False))
df_ca.loc[bool_sr, 'Position_clean'] = SR
# ds
bool_ds = (df_ca['Position_clean'].str.contains('Specialist|Analyst', case = False)) &\
            (df_ca['Position_clean'].str.contains('data science', case = False))
df_ca.loc[bool_ds, 'Position_clean'] = DS

In [14]:
# check 
df_ca[bool_2][['Position', 'Position_clean']]

,Position,Position_clean
4,"Senior Manager, Data Science",Lead Data Scientist
75,Remote PT Data Science Mentor,NaN
204,"Director, Data Science",Lead Data Scientist
210,"Director, Data Science - AIML Coe",Lead Data Scientist
286,Machine Learning Artificial Intelligence Inter...,Data Scientist Intern
363,"Senior Manager, Data Science",Lead Data Scientist
393,"Senior Manager, Data Science",Lead Data Scientist
395,"Senior Manager, Data Science",Lead Data Scientist
459,Data Science Senior Manager,Lead Data Scientist
474,Data Science Senior Manager,Lead Data Scientist


In [15]:
## Step 3
# display all 'analytics' roles
bool_3 = df_ca['Position'].str.contains('analytics', case = False)
df_ca[bool_3][['Position', 'Position_clean']]# display

,Position,Position_clean
269,"Assistant Manager (Senior Data Analyst), Decis...","Assistant Manager (Senior Data Analyst), Decis..."
290,Manager Data Analytics,Manager Data Analytics
420,"Data Scientist, Marketing & Web Analytics","Data Scientist, Marketing & Web Analytics"
435,Lead Data Scientist (Supply Chain Analytics),Lead Data Scientist (Supply Chain Analytics)
456,"Specialist System Design, Data Scientist Machi...","Specialist System Design, Data Scientist Machi..."
506,"Data Scientist, Video Analytics","Data Scientist, Video Analytics"
509,"Data Scientist, Advanced Analytics","Data Scientist, Advanced Analytics"
513,Senior Data Scientist BI Analytics,Senior Data Scientist BI Analytics
519,Senior Data Engineer - Analytics,Senior Data Engineer - Analytics
533,Senior Data Scientist – BI Analytics,Senior Data Scientist – BI Analytics


In [16]:
# replace 'engineer/PM' & 'analytics' roles with NaN
bool_eng = df_ca['Position_clean'].str.contains('engineer|product manager|backend', case = False) & \
           df_ca['Position_clean'].str.contains('analytics', case = False)
df_ca.loc[bool_eng , 'Position_clean'] = rm

In [17]:
df_ca[bool_eng][['Position', 'Position_clean']]

,Position,Position_clean
519,Senior Data Engineer - Analytics,NaN
562,"Senior Software Engineer, Data Analytics",NaN
647,Senior Software Engineer-Data Analytics,NaN
720,Analytics Engineering Senior Specialist,NaN
728,Analytics DevOps Senior Engineer,NaN
743,"Senior Product Manager, Retail Analytics (EST)",NaN
887,"Intermediate/Senior Backend Developer, Analyti...",NaN


In [18]:
## replace 
# LDS
bool_lds = (df_ca['Position_clean'].str.contains('Manage|Direct|Lead|VP|head', case = False)) &\
           (df_ca['Position_clean'].str.contains('Analytics|Data scientist', case = False))
df_ca.loc[bool_lds, 'Position_clean'] = LDS
# SR
bool_sr = (df_ca['Position_clean'].str.contains('senior|sr', case = False)) &\
           (df_ca['Position_clean'].str.contains('Analytics|Data scientist', case = False))
df_ca.loc[bool_sr, 'Position_clean'] = SR
# DS
bool_ds = (df_ca['Position_clean'].str.contains('analyst|specialist', case = False)) &\
           (df_ca['Position_clean'].str.contains('Analytics|Data scientist', case = False))
df_ca.loc[bool_ds, 'Position_clean'] = DS
# check
df_ca[bool_3][['Position', 'Position_clean']]

,Position,Position_clean
269,"Assistant Manager (Senior Data Analyst), Decis...",Lead Data Scientist
290,Manager Data Analytics,Lead Data Scientist
420,"Data Scientist, Marketing & Web Analytics","Data Scientist, Marketing & Web Analytics"
435,Lead Data Scientist (Supply Chain Analytics),Lead Data Scientist
456,"Specialist System Design, Data Scientist Machi...",Data Scientist
506,"Data Scientist, Video Analytics","Data Scientist, Video Analytics"
509,"Data Scientist, Advanced Analytics","Data Scientist, Advanced Analytics"
513,Senior Data Scientist BI Analytics,Senior Data Scientist
519,Senior Data Engineer - Analytics,NaN
533,Senior Data Scientist – BI Analytics,Senior Data Scientist


In [19]:
## Step 4
bool_ml = df_ca['Position'].str.contains('machine learn', case = False, na = False) &\
         ( df_ca['Position'].str.contains('data', case = False, na = False) |\
           df_ca['Position'].str.contains('ML', case = True, na = False) )
df_ca[bool_ml][['Position', 'Position_clean']]

,Position,Position_clean
286,Machine Learning Artificial Intelligence Inter...,Data Scientist Intern
450,"Senior Data Scientist, Machine Learning",Senior Data Scientist
456,"Specialist System Design, Data Scientist Machi...",Data Scientist
547,Data Scientist/ Machine learning Engineer,Data Scientist/ Machine learning Engineer
548,Data Scientist/ Machine learning Engineer,Data Scientist/ Machine learning Engineer
549,Data Scientist/ Machine learning Engineer,Data Scientist/ Machine learning Engineer
697,"Spécialiste, System Design, Data Scientist Mac...","Spécialiste, System Design, Data Scientist Mac..."
718,Data Scientist/Machine Learning Engineer,Data Scientist/Machine Learning Engineer
919,Data Scientist – Machine Learning,Data Scientist – Machine Learning
1025,Machine Learning Engineer/Data Scientist,Machine Learning Engineer/Data Scientist


In [20]:
# replace special case: DS/ML engineer
SPECIAL = ['Data Scientist/', 'Engineer/Data Scientist']
cond = '|'.join(SPECIAL)
df_ca.loc[df_ca['Position'].str.contains(cond, case = False, na = False), 'Position_clean'] = DS

In [21]:
# replace 'developer/engineering' & 'ML/machine learning' roles w NaN
bool_eng2 = df_ca['Position_clean'].str.contains('engineer|develop', case = False) & \
           df_ca['Position_clean'].str.contains('ML|machine learn|AI|artificial intelli', case = False) 
df_ca[bool_eng2][['Position', 'Position_clean']]
df_ca.loc[ bool_eng2 , 'Position_clean'] = rm

In [22]:
# check 
df_ca[bool_ml][['Position', 'Position_clean']]

,Position,Position_clean
286,Machine Learning Artificial Intelligence Inter...,Data Scientist Intern
450,"Senior Data Scientist, Machine Learning",Senior Data Scientist
456,"Specialist System Design, Data Scientist Machi...",Data Scientist
547,Data Scientist/ Machine learning Engineer,Data Scientist
548,Data Scientist/ Machine learning Engineer,Data Scientist
549,Data Scientist/ Machine learning Engineer,Data Scientist
697,"Spécialiste, System Design, Data Scientist Mac...","Spécialiste, System Design, Data Scientist Mac..."
718,Data Scientist/Machine Learning Engineer,Data Scientist
919,Data Scientist – Machine Learning,Data Scientist – Machine Learning
1025,Machine Learning Engineer/Data Scientist,Data Scientist


In [23]:
# replace

# Senior ds
bool_sr = df_ca['Position_clean'].str.contains('senior|sr', case = False, na = False) &\
        (df_ca['Position_clean'].str.contains('ML', case = True, na = False) |\
        df_ca['Position_clean'].str.contains('machine learn', case = False, na = False) )
df_ca.loc[bool_sr, 'Position_clean'] = SR

# ds
bool_ds =  df_ca['Position_clean'].str.contains('scien', case = False, na = False) &\
        (df_ca['Position_clean'].str.contains('ML', case = True, na = False) |\
        df_ca['Position_clean'].str.contains('machine learn', case = False, na = False) )
df_ca.loc[bool_ds, 'Position_clean'] = DS

In [24]:
## Step 5 : other special cases
df_ca.loc[df_ca['Position'] == 'Programmer/Developer Lead - Sr Data Analyst - Visualization', 'Position_clean'] = SR
df_ca.loc[df_ca['Position'] == 'Senior Statistical Programmer/Statistical Scientist', 'Position_clean'] = SR
df_ca.loc[df_ca['Position'] == 'Junior Statistician/Database Specialist (Research Institute)', 'Position_clean'] = JR
df_ca.loc[df_ca['Position'] == 'Data Analyst (SQL/Python/AI)', 'Position_clean'] = DS

In [25]:
## Step 6
bool_notDS = df_ca['Position_clean'].str.contains('Data Scientist', case = False) == False
#df_ca[bool_notDS]['Position_clean'].iloc[340:360]
df_ca.loc[bool_notDS, 'Position_clean'] = rm

In [26]:
df_ca[bool_notDS]['Position'].unique()

array(['Data Analyst',
       'Automotive Software Data Analyst / Tester - Remote',
       'Junior Data Analyst',
       'Optimization Operational Research Scientist', 'Junior Scientist',
       'Senior Database Developer', 'Master Data Analyst / SAP (Junior)',
       'Data Analyst - Junior', 'Data Analyst (Remote)',
       'Remote Cloud Database developer (Python)',
       'Reporting Consultant & Data Analyst / Cognos Expert',
       'Flight Data Analyst', 'Junior Software Developer',
       'Product Data Analyst', 'Data Analyst II',
       'Entry Level Data Analyst (Bilingual French & English)',
       'Jr. Data Analyst', 'Digital Data Analyst',
       'Data Analyst, Security - 297742', 'Tech - Data Analyst',
       'Data Analyst, Product',
       'Data Analyst Intern, Global Investments IT',
       'Data Analyst - Level 1 x2 (Contract)',
       'Student Intern, Customer Data Analyst',
       'Data Analyst and Reporting Specialist', 'Game Data Analyst',
       'Master Data Analyst /V

In [27]:
# check 
a = df_ca[df_ca['Position'].str.contains('Data Scientist', case = False) == False]
a[['Position', 'Position_clean']]

,Position,Position_clean
1,Data Analyst,NaN
2,Data Analyst,NaN
4,"Senior Manager, Data Science",Lead Data Scientist
5,Automotive Software Data Analyst / Tester - Re...,NaN
6,Junior Data Analyst,NaN
7,Junior Data Analyst,NaN
9,Optimization Operational Research Scientist,NaN
11,Data Analyst,NaN
12,Junior Scientist,NaN
13,Data Analyst,NaN


In [28]:
# check no. of unique data scientist listings left after removing irrelevant rows
df_ca[df_ca['Position_clean'].str.contains('Data Scientist', case = False) == True]['Position'].unique().shape

(121,)

In [29]:
df_ca['Position'].unique().shape

(615,)

## Out of 615 *unique* Canada listings, get 121 *unique* data scientist (includes: 6 intern, 1 Jr.,  33 Sr.,  19 Lead,  62 data scientist) listings

In [30]:
bool_lds = df_ca['Position_clean'].str.contains('lead|principal|manage|direct|VP|head', case = False, na = False)
#df_ca[bool_lds]['Position'].unique().shape 
df_ca.loc[bool_lds, 'Position_clean'] = LDS

In [31]:
bool_sr = df_ca['Position_clean'].str.contains('senior|sr', case = False, na = False)
#df_ca[bool_sr]['Position'].unique().shape 
df_ca.loc[bool_sr, 'Position_clean'] = SR

In [32]:
bool_jr = df_ca['Position_clean'].str.contains('junior|jr|early|new grad|entry', case = False, na = False)
df_ca[bool_jr]['Position'].unique().shape 
df_ca.loc[bool_jr, 'Position_clean'] = JR

In [51]:
bool_remain = (df_ca['Position_clean'].str.contains('senior', case = False, na = False) == False) &\
                (df_ca['Position_clean'].str.contains('junior', case = False, na = False) == False) &\
                (df_ca['Position_clean'].str.contains('lead', case = False, na = False) == False) &\
                (df_ca['Position_clean'].str.contains('intern', case = False, na = False) == False) &\
                df_ca['Position_clean'].notna()
df_ca.loc[bool_remain, 'Position_clean'] = DS

In [54]:
# check 
df_ca['Position_clean'].unique()
#df_ca[['Position', 'Position_clean']]

array(['Data Scientist', nan, 'Lead Data Scientist',
       'Data Scientist Intern', 'Senior Data Scientist',
       'Junior Data Scientist'], dtype=object)

In [55]:
# write to csv
df_ca.to_csv('elisa_position_ca.csv')

In [56]:
df = pd.read_csv('elisa_position_ca.csv', index_col = 0)
df.shape

(1258, 13)

In [59]:
df.tail(10)

,Company,Description,Glassdoor Salary Estimate,Industry,Location,Position,Rating,Revenue,Sector,Size,Type,Year Founded,Position_clean
1248,Vigorate Digital Solutions,Who We Are\r\r\nVigorate Digital is made up of...,NaN,NaN,Toronto,Marketing Data Analyst,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1249,Cascadia Scientific,Cascadia Scientific provides a SaaS platform i...,NaN,NaN,Surrey,Jr. Data Analyst,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1250,Tandempark Inc.,TandemPark Inc. is a maturing startup company ...,NaN,NaN,Innisfil,Jr. Data Analyst,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1251,Loblaw Companies Limited,"Location:\r\r\n1 Presidents Choice Circle, Bra...",NaN,Vehicle Dealers,Brampton,Marketing Data Analyst,3.6,$10+ billion (USD),Retail,10000+ Employees,Company - Public,NaN,NaN
1252,Moka,The Role\r\r\nYou are passionate about consume...,NaN,Social Assistance,Montreal,Marketing Data Analyst,3.2,Unknown / Non-Applicable,Non-Profit,201 to 500 Employees,Nonprofit Organization,1978.0,NaN
1253,Napoleon Products,Job Details\r\r\nDescription\r\r\nTitle: Busin...,NaN,NaN,Barrie,BI Data Analyst,3.0,Less than $1 million (USD),NaN,1001 to 5000 Employees,Company - Private,NaN,NaN
1254,Hudson Rouge,"WHO IS GTB?\r\r\n\r\r\nA global, industry-lead...",NaN,Advertising & Marketing,Toronto,Marketing Data Analyst,3.6,Less than $1 million (USD),Business Services,51 to 200 Employees,Company - Private,NaN,NaN
1255,Moka,The Role\r\r\n\r\r\nYou are passionate about c...,NaN,Computer Hardware & Software,Montreal,Marketing Data Analyst,4.6,Unknown / Non-Applicable,Information Technology,201 to 500 Employees,Company - Private,2014.0,NaN
1256,Stingray,Are you ready to join a team of marketing and ...,NaN,NaN,Montreal,Marketing Data Analyst,NaN,Less than $1 million (USD),NaN,1 to 50 Employees,Company - Private,NaN,NaN
1257,7D Surgical,At 7D Surgical you will find a vibrant team of...,NaN,NaN,Toronto,Data Analyst Intern,NaN,Less than $1 million (USD),NaN,1 to 50 Employees,Company - Private,NaN,NaN
